<a href="https://colab.research.google.com/github/wdconinc/practical-computing-for-scientists/blob/master/Lectures/lecture22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lecture #22

In [0]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
import scipy.integrate as ig

##In our last episode

* 2nd order ODEs
  * Vectorizing Runge-Kutta
  * Adaptive step Runge-Kutta
  * Hermite's equation
  * Air drag
  * The physical pendulum
  
Regardless of whether we use fixed step or adaptive step Runge-Kutta ODE solvers, we only obtain the values of the solution at discrete points. Can we determine the value of the solution at arbitrary points in between? We will use interpolation routines for this!

### Vectorizing `solve_rk4` and the physical pendulum

In [0]:
def vsolve_rk4(f,t,y0):
    y0 = np.asarray(y0)
    t = np.asarray(t)
    y = np.zeros((len(t), len(y0)))
    y[0] = y0
    for i in range(0,len(t)-1):
        h = t[i+1] - t[i]
        k1 = h*f(y[i], t[i])
        k2 = h*f(y[i] + k1/2.0, t[i] + h/2.0)
        k3 = h*f(y[i] + k2/2.0, t[i] + h/2.0)
        k4 = h*f(y[i] + k3, t[i]+h)
        y[i+1] = y[i] + 1.0/6.0 * (k1 + 2*k2 + 2*k3 + k4)
    return y

In [0]:
def fpend(y,t):
    yprime = np.zeros_like(y)
    yprime[0] = y[1]
    yprime[1] = -15*np.sin(y[0])
    return yprime

y0 = [math.pi/4.0, 0]

t = np.linspace(0, 3)
ypend = vsolve_rk4(fpend,t,y0)
plt.plot(t, ypend[:,0], "-og", ms = 3)
plt.plot([2*math.pi/math.sqrt(15)]*10, np.linspace(-.8,.8,10), "r")
plt.annotate("$T_{ideal}=2\pi/\sqrt{15}$", (1.622,.8), (0.5,0.8), arrowprops = {"arrowstyle": '->'})

##Using `scipy.integrate.odeint`

In [0]:
yodeint = ig.odeint(fpend, y0, t)
plt.plot(t, yodeint[:,0], '-or', ms = 3)

##Interpolation

Imagine that I know the value of a function $y_i=f(x_i)$ at a set of points $x_i$, but I don't know the function itself. For example, perhaps I obtained $\{x_i,y_i\}$ from numerically solving a differential equation. With just that information, it's hard to utilize some of the techniques we've developed:

* Taking derivatives
* Doing integrals
* Finding roots and minima/maxima
* Plotting the points as a curve

_Interpolation_ refers to any procedure which gives us a function $y(x)$ that estimates the true value of the function $f(x)$ for values of $x$ in the range $x_0< x < x_N$. Generally, we want a procedure that goes through each of our points, i.e., one that satisfies $y(x_i)=y_i$, and we want $y(x)$ to be nicely behaved in between the points.


###Polynomial interpolation

You know that any two points define (i.e., can be used to get the equation of) a straight line, and may also know that any three points define a parabola. In general, if I have $N$ points $\{x_i,y_i\}$, I can find an $N-1$ order polynomial that goes through each of the points. This polynomial can be written as:

\begin{align} y(x)   &=   \frac{(x-x_1)(x-x_2)\ldots(x-x_{N-1})}{(x_0-x_1)(x_0-x_2)\ldots(x_0-x_{N-1})}y_0 \\
&+ \frac{(x-x_0)(x-x_2)\ldots(x-x_{N-1})}{(x_1-x_0)(x_1-x_2)\ldots(x_1-x_{N-1})}y_1\\
&+ \ldots \\
&+  \frac{(x-x_0)(x-x_1)\ldots(x-x_{N-1})}{(x_{N-1}-x_0)(x_{N-1}-x_2)\ldots(x_{N-1}-x_{N-2})}y_{N-1}
\end{align}

This is _Lagrange's formula_ using Lagrange's _cardinal functions $h_i(x)$_. Note, it's kind of clever. If I pick $x=x_0$ the first term is just $y_0$ and _all other terms are zero_. If I pick $x=x_1$ then the second term is $y_1$ and all other terms are zero. And so on. This equation clearly satisfies $y(x_i)=y_i$, it's differentiable and smooth because it's a polynomial, and looks pretty nice.

####A special case: linear interpolation

If I have two points, $x_i$ and $x_{i+1}$, Lagrange's equation reduces to

\begin{align} y(x) & = \frac{(x-x_{i+1})}{(x_i-x_{i+1})}y_{i} + \frac{(x-x_{i})}{(x_{i+1}-x_{i})}y_{i+1} \\
&= \frac{(x_{i+1}-x)}{(x_{i+1}-x_{i})}y_{i} + \frac{(x-x_{i})}{(x_{i+1}-x_{i})}y_{i+1}\\
&= A y_{i} + B y_{i+1}
\end{align}

###The good, the bad and the ugly of polynomial interpolation.

__The good:__ Continuous, smooth, goes through the points, straightforward

__The bad:__ If I have 100 points, I'd be computing terms with $x^{99}$

__The ugly:__

In [0]:
import scipy.interpolate as ip
x = [i for i in range(1,20)]
y = [i for i in range(1,20)]
fl = ip.lagrange(x, y)
xx = np.linspace(0, 20, 200)
plt.plot(x, y, 'or', xx, fl(xx), '-b', ms = 3)

In [0]:
X1 = np.linspace(-5, +5, 25)
X2 = np.linspace(-5, +5, 2500)
f2 = lambda x: 1 / (1 + x**2)
Y1 = f2(X1)
plt.plot(X1, Y1, 'ok')
f2_interp = ip.lagrange(X1, Y1)
plt.plot(X2, f2_interp(X2), '-r')
plt.ylim(-2,2)

##Splines

The idea behind splines is to do local interpolation, say between $x_i$ and $x_{i+1}$, with a low order polynomial and then stitch together the local interpolations to get $y(x)$. The stitching certainly entails making sure the function is continuous, but could also involve conditions on the first and higher order derivatives. 

###Cubic splines

Linear interpolation is bad/unrealistic because the first derivative is discontinuous. This makes the function look unphysical, since it is, and also could cause problems for routines that rely on the first or, worse, second derivative. Cubic splines attempt to solve this problem by interpolating between two points $\{x_i,y_i\}$ and $\{x_{i+1},y_{i+1}\}$ with a cubic equation 

$$y(x)=a+b x+c x^2 + d x^3$$

Since the equation is a cubic, the second derivative will (in general) be a linear function and the first will be a quadratic. The curve will have a nice smooth appearance. 

A cubic has 4 unknowns so we need two more constraints in addition to the requirement that the curve passes through $\{x_i,y_i\}$ and $\{x_{i+1},y_{i+1}\}$. They come from requiring that the derivative $y_i'=y(x_i)$ is the same when computed in adjacent intervals $[x_{i-1},x_{i}]$ and $[x_{i},x_{i+1}]$.  We can do this for every point except the two endpoints $\{x_0,y_0\}$ and $\{x_{N-1},y_{N-1}\}$, so we don't have quite enough information



###Interpolating the physical pendulum solution

In [0]:
import scipy.interpolate as ip

In [0]:
# [shift-enter] tells you that it does a cubic spline interpolation by default
fip = ip.InterpolatedUnivariateSpline(t, yodeint[:,0])
tt = np.linspace(0, 3, 200)
plt.plot(t, yodeint[:,0], 'ok', tt, fip(tt), "-r", ms = 5)

##Little project: plot the period as a function of initial angle

* Loop over initial angles
* For each use odeint to solve the diff. eq.
* Then, make an interpolated function
* Then, find the position of the first maximum with `scipy.optimize.minimize`

In [0]:
import scipy.optimize as op

In [0]:
t = np.linspace(0, 3, 30)
tt = np.linspace(0, 3, 300)
thetas = []
periods = []
for i in range(1, 10):
    theta_0 = i*0.1*math.pi/2.0
    thetas.append(theta_0)
    y0 = [theta_0, 0]
    y = ig.odeint(fpend, y0, t)
    plt.plot(t, y[:,0], 'o', ms = 3)
    f = ip.InterpolatedUnivariateSpline(t, y[:,0])
    plt.plot(tt, f(tt))
    fneg = lambda x: -1*f(x)
    res = op.minimize_scalar(fneg, (1.25,1.8,2.25), method = 'golden')
    res = op.minimize_scalar(fneg, (1.25,1.8,2.25), method = 'brent')
    print(theta_0, res.x)
    periods.append(res.x / (2*math.pi/math.sqrt(15)))

In [0]:
plt.plot(thetas, periods, "-og")
plt.xlabel('initial angle - $\\theta_0$ (radians)')
plt.ylabel('$T/T_{ideal}$')